In [1]:
!pip install pycocotools
!pip install ultralytics
!pip install -U ipywidgets
!pip install clearml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 7.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 kB 14.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 3.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 46.8 MB/s eta 0:00:0000:01
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.9
    Uninstalling widgetsnbextension-3.6.9:
      Successfully uninstalled widgetsnbextension-3.6.9
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab_widgets 3.0.11
    Uninstalling jupyterlab_widgets-3.0.11:
      Successfully uninstalled jupyterlab_widgets-3.0.11
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1

In [3]:
!clearml-init

from clearml import Task

task = Task.init(project_name="DeepFashion", task_name="DeepFashion_YOLOv8n_seg191124")

ClearML SDK setup process

Please create new clearml credentials through the settings page in your `clearml-server` web app (e.g. http://localhost:8080//settings/workspace-configuration) 
Or create a free account at https://app.clear.ml/settings/workspace-configuration

In settings page, press "Create new credentials", then press "Copy to clipboard".

Paste copied configuration here:
^C
Traceback (most recent call last):
  File "/opt/conda/bin/clearml-init", line 8, in <module>
    sys.exit(main())
  File "/opt/conda/lib/python3.10/site-packages/clearml/cli/config/__main__.py", line 82, in main
    api_input = input()
KeyboardInterrupt
ClearML Task: created new task id=b6607c382b684248866e10ccf42eb834
2024-11-19 14:51:26,494 - clearml.Repository Detection - WARNING - Jupyter Notebook auto-logging failed, could not access: /kaggle/working/__notebook_source__.ipynb
2024-11-19 14:51:26,516 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.

/opt/conda/lib/python3.10/site-packages/clearml/utilities/process/mp.py:629: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.



ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [ ]:
# import os
# import json
# from PIL import Image
# from pycocotools.coco import COCO
# from shapely.geometry import Polygon, MultiPolygon


# def yolo_format(coco_annotation_path, image_folder, output_label_folder):
#     coco = COCO(coco_annotation_path)
    
#     for img_id in coco.getImgIds():
#         img_info = coco.loadImgs(img_id)[0]
#         img_filename = img_info['file_name']
#         img_width, img_height = img_info['width'], img_info['height']
        
#         ann_ids = coco.getAnnIds(imgIds=img_id)
#         anns = coco.loadAnns(ann_ids)
        
#         label_filename = os.path.splitext(img_filename)[0] + '.txt'
#         os.makedirs(output_label_folder, exist_ok=True)
#         label_path = os.path.join(output_label_folder, label_filename)
        
#         with open(label_path, 'w') as label_file:
#             for ann in anns:
#                 if 'segmentation' in ann and ann['segmentation']:
#                     if ann['iscrowd'] == 0:
#                         class_id = ann['category_id'] - 1
                        
#                         segmentation = ann['segmentation'][0]
#                         poly = Polygon([(segmentation[i], segmentation[i+1]) for i in range(0, len(segmentation), 2)])
                        
#                         normalized_coords = []
#                         for x, y in poly.exterior.coords:
#                             normalized_coords.append(x / img_width)
#                             normalized_coords.append(y / img_height)
                        
#                         label_file.write(f"{class_id} " + " ".join(map(str, normalized_coords)) + "\n")

In [ ]:
# train_coco_path = '/kaggle/input/df2-train-img/ann_train.json'
# train_image_folder = '/kaggle/input/df2-train-img/train_image/image'
# train_label_folder = '/kaggle/working/train_labels'

# val_coco_path = '/kaggle/input/df2-img-val/ann_val.json'
# val_image_folder = '/kaggle/input/df2-img-val/image/image'
# val_label_folder = '/kaggle/working/val_labels'

# yolo_format(train_coco_path, train_image_folder, train_label_folder)
# yolo_format(val_coco_path, val_image_folder, val_label_folder)

# Model 

In [9]:
import yaml


cat_names = [
    'short sleeved shirt', 'long sleeved shirt', 'short sleeved outwear', 'long sleeved outwear',
    'vest', 'sling', 'shorts', 'trousers', 'skirt', 'short sleeved dress',
    'long sleeved_dress', 'vest dress', 'sling dress'
]

config = {
    "path": "/kaggle/input/df2-yolo",
    'val': "/kaggle/input/df2-yolo/val/images", 
    'train': "/kaggle/input/df2-yolo/val/images",
    'nc': 13,
    'names': cat_names
}

# Save the dictionary as a YAML file
with open('config.yaml', 'w') as file:
    yaml.dump(config, file, default_flow_style=False)

print("YAML configuration saved as 'config.yaml'")

YAML configuration saved as 'config.yaml'


In [10]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda:0


In [11]:
import os

os.makedirs("/kaggle/working/cache", exist_ok=True)

In [12]:
from ultralytics import YOLO

# Initialize a YOLO model (choose a pre-trained YOLOv8 model for segmentation)
model = YOLO('yolov8n-seg.pt')  


args = dict(
    data='/kaggle/working/config.yaml',
    cache="/kaggle/working/cache",
    save=True,
    epochs=40,
    warmup_epochs=4,
    val=False,
    batch=128,
    imgsz=640,
    freeze=5,
    lr0=0.002,
    patience=5,
    name='yolo_segmentation_model',
    device=device,
    plots=True,
)

task.connect(args)
results = model.train(**args, amp=True)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.74M/6.74M [00:00<00:00, 21.2MB/s]


2024-11-19 15:09:24,198 - clearml.model - INFO - Selected model id: 19ee00e9d78548e2883573fd72b9f071
Ultralytics 8.3.34 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/kaggle/working/config.yaml, epochs=40, time=None, patience=5, batch=128, imgsz=640, save=True, save_period=-1, cache=/kaggle/working/cache, device=cuda:0, workers=8, project=None, name=yolo_segmentation_model, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=5, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=False, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_mask

100%|██████████| 755k/755k [00:00<00:00, 4.15MB/s]


Overriding model.yaml nc=80 with nc=13

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

...


      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      14/40      13.7G     0.5992      1.392      1.036      1.033         94        640: 100%|██████████| 252/252 [06:21<00:00,  1.51s/it]



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      15/40      13.7G     0.5906       1.37      1.028       1.03        502        640:  39%|███▉      | 99/252 [02:29<03:50,  1.51s/it]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.

      15/40      13.7G     0.5906       1.37      1.028       1.03        502        640:  39%|███▉      | 99/252 [02:30<03:51,  1.52s/it]


KeyboardInterrupt: 